### 予測モデルの仕組みをわかりやすく説明します！

#### どんなことを予測するの？
このモデルは、**「当月末の限界利益がどのくらいになりそうか」**を、日次のデータを使って予測するものです。

#### どうやって予測するの？

1.  **日次のデータを使うよ！**
    *   毎日の限界利益データ（「MP」という名前だよ）と、確定値、A、B、C、Dの値を使います。
    *   これらのデータから、「当月末にどれくらいの限界利益があるか」のヒントを探すんだ。

2.  **２つの限界利益を足し算するよ！**
    当月末の限界利益を予測するとき、大きく分けて２つの部分に分けて考えます。

    *   **「もう決まっている限界利益（Confirmed）」**:
        これは、「確定MP」と「AMP（A）」を合わせた限界利益です。これはほぼ確実に手に入る限界利益だね！

    *   **「これから増えそうな限界利益（Additional）」**:
        これは、「まだ確定していないけど、これまでの経験から考えると、これくらいは増えそうだね」という限界利益です。
        この部分を予測するために、特別な計算を使います。特に「BCD」という種類の限界利益（「B」「C」「D」という３つの種類の限界利益を足したものだよ）が、どのくらい「これから増えそうな限界利益」に変わるかを過去のデータから学ぶんだ。

3.  **合計するよ！**
    最後に、「もう決まっている限界利益」と「これから増えそうな限界利益」を足し合わせると、当月末の限界利益の予測が出てきます。

#### まとめると...

このモデルは、日次の限界利益データ（確定値、A、B、C、D）を使って、**「すでに決まっている限界利益」**と**「過去の傾向から見て、これから増えそうな限界利益」**を別々に計算し、それらを合計することで、**「当月末の限界利益がどのくらいになりそうか」**を教えてくれる賢い予測屋さんなんです！

### 予測モデルの仕組みをもう少し詳しく説明します！

先ほどは、当月末の限界利益を「もう決まっている限界利益」と「これから増えそうな限界利益」の2つに分けて考えると説明しましたね。

「これから増えそうな限界利益（Additional）」を予測するために、このモデルでは主に2つの方法を使っています。

#### 1. 「ランダムフォレスト」という賢い予測屋さん

*   **ランダムフォレストって何？**
    これは、たくさんの小さな「予測の木」を集めて、みんなで相談して答えを出すようなものです。例えば、「この状況なら限界利益がこれくらい増えるんじゃない？」という予測を、それぞれが考えて、最後に一番良い答えを決めるイメージです。

*   **どうやって使うの？**
    過去のデータ（特に「B」「C」「D」といった種類の限界利益や、それが何月の何日目に起こったか）をこのたくさんの「予測の木」たちに教えてあげます。すると、「B」がいくらで「C」がいくらで「D」がいくらで、何月の何日目だったら、どれくらい「これから増えそうな限界利益」になるかを賢く予測してくれるんです。

#### 2. 「月別・日別の転換率」で予測

*   **転換率って何？**
    これは、「これくらいの限界利益（BCD）があったら、過去の経験から見て、だいたい何パーセントくらいが当月末に実際に限界利益になるだろう？」という割合のことです。

*   **どうやって使うの？**
    例えば、「1月の15日だったら、BCDの限界利益があったら過去のデータではだいたい70%が『これから増えそうな限界利益』になっていたな」というように、月ごと・日ごとにその割合（転換率）を計算しておきます。
    そして、新しい予測をするときには、その月とその日の転換率を使って「これくらいは増えるだろう」と予測します。

#### 最終的な予測

このモデルでは、この「ランダムフォレスト」の予測と「月別・日別の転換率」を使った予測を、良いバランスで組み合わせています。こうすることで、片方だけの予測よりも、もっと正確な「これから増えそうな限界利益」を出すことができるんですよ。

これに「もう決まっている限界利益」を足し合わせることで、当月末の限界利益を最終的に予測しています。

# MP Current Month-End Prediction Model
## 日次データから当月末MP予測モデル

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded!')

## 2. データの読み込み

In [ ]:
try:
    from google.colab import files
    print('Upload CSV file:')
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
except:
    filename = 'progress_data.csv'
    print(f'Using: {filename}')

In [ ]:
try:
    df = pd.read_csv(filename, encoding='utf-8-sig')
except:
    df = pd.read_csv(filename, encoding='shift_jis')

print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
df.head()

## 3. データ前処理（日次データを使用）

In [ ]:
# 元のカラム名を保存
original_columns = df.columns.tolist()
print(f'Original columns: {original_columns}')

# 日付処理
df['date'] = pd.to_datetime(df.iloc[:, 0], errors='coerce')
# NaT（無効な日付）を含む行を削除
df = df.dropna(subset=['date']).copy()

# 年月情報を追加
df['year_month'] = df['date'].dt.to_period('M')
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

print(f'\nDaily data points: {len(df)}')

In [ ]:
# カラム名を変更（確定、A、B、C、Dに対応）
col_mapping = {
    original_columns[1]: 'MP',
    original_columns[2]: 'Kakutei',  # 確定MP
    original_columns[3]: 'A',        # AMP
    original_columns[4]: 'B',        # BMP
    original_columns[5]: 'C',        # CMP
    original_columns[6]: 'D'         # DMP
}
print(f'Mapping: {col_mapping}')

df = df.rename(columns=col_mapping)

# 数値変換
for col in ['MP', 'Kakutei', 'A', 'B', 'C', 'D']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f'\nColumns after mapping: {df.columns.tolist()}')
df.head()

In [ ]:
# 各月の月末MPを取得（ターゲット変数として使用）
month_end_mp = df.groupby('year_month')['MP'].last().to_dict()

# 日次データに当月末MPを追加
df['Current_Month_End_MP'] = df['year_month'].map(month_end_mp)

# 訓練データ作成（月末以外の日のデータを使用）
# 月末は除外（予測対象なので）
month_end_dates = df.groupby('year_month')['date'].max()
df['is_month_end'] = df.apply(lambda x: x['date'] == month_end_dates[x['year_month']], axis=1)

# 訓練データ：月末を除く日次データ
train_data = df[~df['is_month_end']].copy()

print(f'Training data points: {len(train_data)} days')
print(f'Unique months: {train_data["year_month"].nunique()}')

if len(train_data) > 0:
    train_data['Confirmed'] = train_data['Kakutei'] + train_data['A']
    train_data['Additional'] = train_data['Current_Month_End_MP'] - train_data['Confirmed']
    train_data['BCD_Total'] = train_data['B'] + train_data['C'] + train_data['D']
    
    # 転換率計算（BCDがAdditionalにどれくらい変わるか）
    train_data['Conversion_Rate'] = np.where(
        train_data['BCD_Total'] > 0,
        train_data['Additional'] / train_data['BCD_Total'] * 100,
        0
    )
    
    print('Calculations complete!')
    print(train_data[['date', 'MP', 'Confirmed', 'BCD_Total', 'Current_Month_End_MP']].head(10))
else:
    print('ERROR: No training data!')

## 4. 月別・日別転換率の分析

In [ ]:
# 月別転換率
monthly_stats = train_data.groupby('month').agg({
    'Conversion_Rate': ['mean', 'std', 'count']
}).round(1)
monthly_stats.columns = ['Avg_Rate', 'Std', 'Count']

print('Monthly BCD Conversion Rate')
print('='*50)
month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for m in range(1, 13):
    if m in monthly_stats.index:
        r = monthly_stats.loc[m]
        print(f'{m:2d} ({month_names[m]}): {r["Avg_Rate"]:5.1f}% (n={int(r["Count"])})')
        
display(monthly_stats)

In [ ]:
# 月内の日による転換率の変化を可視化
fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()

for m in range(1, 13):
    month_data = train_data[train_data['month'] == m]
    if len(month_data) > 0:
        daily_avg = month_data.groupby('day')['Conversion_Rate'].mean()
        axes[m-1].plot(daily_avg.index, daily_avg.values, marker='o')
        axes[m-1].set_title(f'{month_names[m]} - Conversion Rate by Day')
        axes[m-1].set_xlabel('Day of Month')
        axes[m-1].set_ylabel('Rate (%)')
        axes[m-1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. モデル構築

In [ ]:
class CurrentMonthMPPredictor:
    def __init__(self):
        self.model = None
        self.monthly_rates = {}
        self.overall_avg_rate = None

    def train(self, data):
        # 月別平均転換率を計算
        for m in range(1, 13):
            sub = data[data['month'] == m]
            if len(sub) > 0:
                self.monthly_rates[m] = sub['Conversion_Rate'].mean()
        self.overall_avg_rate = data['Conversion_Rate'].mean()

        # Random Forestモデルの訓練
        # 特徴量: B、C、D、月、日
        X = data[['B', 'C', 'D', 'month', 'day']]
        y = data['Additional']
        
        self.model = RandomForestRegressor(
            n_estimators=100, 
            max_depth=7,  # 深さを増やして日次データの複雑さに対応
            random_state=42
        )
        self.model.fit(X, y)

        # ハイブリッド予測で評価（predict()と同じロジック）
        pred_additional_rf = self.model.predict(X)
        pred_additional_hybrid = []
        for i, (_, row) in enumerate(data.iterrows()):
            rate = self.monthly_rates.get(int(row['month']), self.overall_avg_rate) / 100
            add_rate = row['BCD_Total'] * rate
            hybrid = pred_additional_rf[i] * 0.7 + add_rate * 0.3
            pred_additional_hybrid.append(hybrid)
        
        pred = data['Confirmed'] + np.array(pred_additional_hybrid)
        
        return {
            'mae': mean_absolute_error(data['Current_Month_End_MP'], pred),
            'r2': r2_score(data['Current_Month_End_MP'], pred)
        }

    def predict(self, kakutei, a, b, c, d, month, day):
        """日次の確定値、A、B、C、Dから当月末のMPを予測"""
        confirmed = kakutei + a
        bcd = b + c + d

        # Random Forestによる予測
        add_rf = self.model.predict([[b, c, d, month, day]])[0]
        
        # 月別転換率による予測
        rate = self.monthly_rates.get(month, self.overall_avg_rate) / 100
        add_rate = bcd * rate
        
        # ハイブリッド予測（RF 70% + 転換率 30%）
        additional = add_rf * 0.7 + add_rate * 0.3

        return {
            'month': month,
            'day': day,
            'confirmed': confirmed,
            'additional': additional,
            'forecast': confirmed + additional,
            'bcd': bcd,
            'rate': rate * 100
        }

print('CurrentMonthMPPredictor defined!')

In [ ]:
predictor = CurrentMonthMPPredictor()
metrics = predictor.train(train_data)

print('Model Trained!')
print(f'MAE: {metrics["mae"]/10000:,.0f} (10K JPY)')
print(f'R2:  {metrics["r2"]:.3f}')

## 6. 予測実行

In [ ]:
def show_prediction(p):
    mn = ['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    print('='*50)
    print(f'{mn[p["month"]]} Day {p["day"]} -> Month-End Forecast')
    print('='*50)
    print(f'Confirmed:  {p["confirmed"]/10000:>10,.0f} (10K)')
    print(f'Additional: {p["additional"]/10000:>10,.0f} (10K)')
    print(f'FORECAST:   {p["forecast"]/10000:>10,.0f} (10K)')
    print(f'Conv Rate: {p["rate"]:.1f}%')
    print('='*50)

In [ ]:
# 最新データで予測
latest = df[~df['is_month_end']].iloc[-1]
print(f'Latest: {latest["date"].strftime("%Y-%m-%d")}')

p = predictor.predict(
    latest['Kakutei'], latest['A'],
    latest['B'], latest['C'], latest['D'],
    month=int(latest['month']),
    day=int(latest['day'])
)
show_prediction(p)

# 実際の月末MPと比較
actual_month_end = latest['Current_Month_End_MP']
print(f'\nActual Month-End MP: {actual_month_end/10000:>10,.0f} (10K)')
print(f'Prediction Error: {abs(p["forecast"] - actual_month_end)/10000:>10,.0f} (10K)')

In [ ]:
# カスタム予測例
# 例: 2月15日時点のデータから2月末を予測
p = predictor.predict(
    kakutei=609289, 
    a=47905097,
    b=5236622, 
    c=3481478, 
    d=18589534,
    month=2,  # 2月
    day=15    # 15日
)
show_prediction(p)

## 7. 月内予測精度の推移

In [ ]:
# 各月の日ごとの予測精度を分析
predictions_df = train_data.copy()
X_pred = predictions_df[['B', 'C', 'D', 'month', 'day']]
predictions_df['Predicted_Additional'] = predictor.model.predict(X_pred)
predictions_df['Predicted_Month_End_MP'] = predictions_df['Confirmed'] + predictions_df['Predicted_Additional']
predictions_df['Prediction_Error'] = abs(predictions_df['Predicted_Month_End_MP'] - predictions_df['Current_Month_End_MP'])

# 月別の予測誤差推移を可視化
fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()
month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for m in range(1, 13):
    month_data = predictions_df[predictions_df['month'] == m]
    if len(month_data) > 0:
        daily_error = month_data.groupby('day')['Prediction_Error'].mean() / 10000
        axes[m-1].plot(daily_error.index, daily_error.values, marker='o', color='coral')
        axes[m-1].set_title(f'{month_names[m]} - Prediction Error by Day')
        axes[m-1].set_xlabel('Day of Month')
        axes[m-1].set_ylabel('MAE (10K JPY)')
        axes[m-1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. 実績値と予測値の比較

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# サンプルとして各月の中旬（15日前後）のデータを抽出して比較
mid_month_data = predictions_df[predictions_df['day'].between(14, 16)].copy()
mid_month_data = mid_month_data.groupby('year_month').first().reset_index()

plt.figure(figsize=(15, 7))
plt.plot(mid_month_data['year_month'].astype(str), 
         mid_month_data['Current_Month_End_MP']/10000, 
         label='Actual Month-End MP', marker='o', linewidth=2)
plt.plot(mid_month_data['year_month'].astype(str), 
         mid_month_data['Predicted_Month_End_MP']/10000, 
         label='Predicted Month-End MP (from mid-month)', marker='x', linewidth=2)

plt.title('Actual vs Predicted Month-End MP (from Mid-Month Data)')
plt.xlabel('Year-Month')
plt.ylabel('MP (10K JPY)')
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

# 統計情報
print('\nMid-Month Prediction Statistics:')
print(f'MAE: {mid_month_data["Prediction_Error"].mean()/10000:,.0f} (10K JPY)')
print(f'Max Error: {mid_month_data["Prediction_Error"].max()/10000:,.0f} (10K JPY)')
print(f'Min Error: {mid_month_data["Prediction_Error"].min()/10000:,.0f} (10K JPY)')

## 9. 月内の日ごとのシミュレーション

In [ ]:
# 特定の月のデータを使って、日ごとの予測がどう変化するかシミュレーション
latest_month = df[df['year_month'] == df['year_month'].max()].copy()
latest_month = latest_month[~latest_month['is_month_end']]

if len(latest_month) > 0:
    simulation_results = []
    
    for _, row in latest_month.iterrows():
        p = predictor.predict(
            row['Kakutei'], row['A'],
            row['B'], row['C'], row['D'],
            month=int(row['month']),
            day=int(row['day'])
        )
        simulation_results.append({
            'date': row['date'],
            'day': row['day'],
            'confirmed': p['confirmed'],
            'additional': p['additional'],
            'forecast': p['forecast'],
            'actual': row['Current_Month_End_MP']
        })
    
    sim_df = pd.DataFrame(simulation_results)
    
    # 可視化
    plt.figure(figsize=(12, 6))
    plt.plot(sim_df['day'], sim_df['forecast']/10000, 
             label='Predicted Month-End MP', marker='o', linewidth=2)
    plt.axhline(y=sim_df['actual'].iloc[0]/10000, 
                color='red', linestyle='--', label='Actual Month-End MP', linewidth=2)
    plt.fill_between(sim_df['day'], 
                     sim_df['confirmed']/10000, 
                     sim_df['forecast']/10000, 
                     alpha=0.3, label='Additional (Predicted)')
    
    plt.title(f'Daily Prediction Progress for {latest_month.iloc[0]["year_month"]}')
    plt.xlabel('Day of Month')
    plt.ylabel('MP (10K JPY)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print('\nDaily Prediction Summary:')
    display(sim_df[['day', 'confirmed', 'additional', 'forecast', 'actual']].head(10))
else:
    print('No data available for simulation')

## 10. 過去の任意の月の推移分析（インタラクティブ）

In [ ]:
# 利用可能な月のリストを表示
available_months = sorted(df['year_month'].unique())
print('Available Months:')
print('='*60)
for i, ym in enumerate(available_months, 1):
    print(f'{i:2d}. {ym}')

print(f'\nTotal: {len(available_months)} months of data')

In [ ]:
# 分析したい月を選択（例：2025年6月なら '2025-06'）
# 上のリストから選んでください
selected_month = '2025-06'  # ← ここを変更してください

# 選択した月のデータを取得
selected_month_period = pd.Period(selected_month, freq='M')
selected_data = df[df['year_month'] == selected_month_period].copy()
selected_data = selected_data[~selected_data['is_month_end']]

if len(selected_data) > 0:
    print(f'Analyzing: {selected_month}')
    print(f'Data points: {len(selected_data)} days')
    print('='*60)
    
    # 日ごとの予測を計算
    simulation_results = []
    
    for _, row in selected_data.iterrows():
        p = predictor.predict(
            row['Kakutei'], row['A'],
            row['B'], row['C'], row['D'],
            month=int(row['month']),
            day=int(row['day'])
        )
        simulation_results.append({
            'date': row['date'],
            'day': row['day'],
            'confirmed': p['confirmed'],
            'additional': p['additional'],
            'forecast': p['forecast'],
            'actual': row['Current_Month_End_MP']
        })
    
    sim_df = pd.DataFrame(simulation_results)
    
    # グラフを2つ作成
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    # グラフ1: 予測値の推移
    ax1.plot(sim_df['day'], sim_df['forecast']/10000, 
             label='Predicted Month-End MP', marker='o', linewidth=2, color='steelblue')
    ax1.axhline(y=sim_df['actual'].iloc[0]/10000, 
                color='red', linestyle='--', label='Actual Month-End MP', linewidth=2)
    ax1.fill_between(sim_df['day'], 
                     sim_df['confirmed']/10000, 
                     sim_df['forecast']/10000, 
                     alpha=0.3, label='Additional (Predicted)', color='lightblue')
    ax1.set_title(f'Daily Prediction Progress for {selected_month}')
    ax1.set_xlabel('Day of Month')
    ax1.set_ylabel('MP (10K JPY)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # グラフ2: 予測誤差の推移
    sim_df['error'] = abs(sim_df['forecast'] - sim_df['actual']) / 10000
    ax2.plot(sim_df['day'], sim_df['error'], 
             marker='o', linewidth=2, color='coral')
    ax2.set_title(f'Prediction Error by Day for {selected_month}')
    ax2.set_xlabel('Day of Month')
    ax2.set_ylabel('Absolute Error (10K JPY)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print('\nPrediction Statistics:')
    print('='*60)
    print(f'Actual Month-End MP:   {sim_df["actual"].iloc[0]/10000:>12,.0f} (10K JPY)')
    print(f'Average Prediction:    {sim_df["forecast"].mean()/10000:>12,.0f} (10K JPY)')
    print(f'Final Day Prediction:  {sim_df["forecast"].iloc[-1]/10000:>12,.0f} (10K JPY)')
    print(f'Average Error (MAE):   {sim_df["error"].mean():>12,.0f} (10K JPY)')
    print(f'Maximum Error:         {sim_df["error"].max():>12,.0f} (10K JPY)')
    print(f'Minimum Error:         {sim_df["error"].min():>12,.0f} (10K JPY)')
    
    print('\nDaily Prediction Summary (First 10 days):')
    display(sim_df[['day', 'confirmed', 'additional', 'forecast', 'actual', 'error']].head(10))
    
    print('\nDaily Prediction Summary (Last 5 days):')
    display(sim_df[['day', 'confirmed', 'additional', 'forecast', 'actual', 'error']].tail(5))
    
else:
    print(f'Error: Data not found for {selected_month}.')
    print('Please select a valid year-month from the list above.')